In [18]:
import numpy as np
import pandas as pd 
import os
from nltk.corpus import stopwords
from textblob import Word
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [19]:
data = pd.read_csv(os.path.join(os.getcwd(),"../static/dataset/emoji_textdata.csv"))
data

,Unnamed: 0,Text,Emotion
0,1,i didnt feel humiliated😔,sadness
1,2,i can go from feeling so hopeless to so damned...,sadness
2,3,im grabbing a minute to post i feel greedy wrong👿,anger
3,4,i am ever feeling nostalgic about the fireplac...,love
4,5,i am feeling grouchy💥,anger
...,...,...,...
15995,15996,i just had a very brief time in the beanbag an...,sadness
15996,15997,i am now turning and i feel pathetic that i am...,sadness
15997,15998,i feel strong and good overall🤩,joy
15998,15999,i feel like this was such a rude comment and i...,anger


In [43]:
data = data[['Text','Emotion']]

In [44]:
from sklearn.model_selection import train_test_split
train, validate = train_test_split(data,test_size=0.1)
train

,Text,Emotion
3188,i should give as charity only what i feel is v...,joy
8214,im very very very very sorry i havent been fee...,joy
7970,i got to feel carefree on the ice with the col...,joy
15378,i feel blank the more it freaks me out😭,sadness
14310,i can remember when cammie was a couple of mon...,anger
...,...,...
11701,i trust you enough to share a pretty humiliati...,joy
4770,i can feel rejected just because someone needs...,sadness
4583,i wonder if they ever feel any pain or sadness...,joy
3238,im not feeling exactly thrilled with standing ...,joy


In [45]:
# def de_repeat(text):
#     pattern = re.compile(r"(.)\1{2,}")
#     return pattern.sub(r"\1\1", text)

# def preprocess(x):
#     x = ' '.join([item for item in str(x).split() if item not in stopwords.words('english')])
#     x = ' '.join(x.lower() for x in x.split())
#     x = ' '.join([Word(word).lemmatize() for word in x.split()])
#     x = ' '.join(de_repeat(x) for x in x.split())
#     return x

In [46]:
from nltk.corpus import stopwords

train['Text'] = train['Text'].apply(lambda x: ' '.join([item for item in str(x).split() if item not in stopwords.words('english')]))
validate['Text'] = validate['Text'].apply(lambda x: ' '.join([item for item in str(x).split() if item not in stopwords.words('english')]))

In [47]:
train['Text'] = train['Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
validate['Text'] = validate['Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train

,Text,Emotion
3188,give charity feel valuable person receiving it😍,joy
8214,im sorry havent feeling well🤩,joy
7970,got feel carefree ice cold air nipping face😊,joy
15378,feel blank freaks out😭,sadness
14310,remember cammie couple months old looking swee...,anger
...,...,...
11701,trust enough share pretty humiliating experien...,joy
4770,feel rejected someone needs sleep😢,sadness
4583,wonder ever feel pain sadness always seem lively😍,joy
3238,im feeling exactly thrilled standing front mir...,joy


In [48]:
import emoji
train['Text'] = train['Text'].apply(lambda x: emoji.demojize(x))
validate['Text'] = validate['Text'].apply(lambda x: emoji.demojize(x))
train

,Text,Emotion
3188,give charity feel valuable person receiving it...,joy
8214,im sorry havent feeling well:star-struck:,joy
7970,got feel carefree ice cold air nipping face:sm...,joy
15378,feel blank freaks out:loudly_crying_face:,sadness
14310,remember cammie couple months old looking swee...,anger
...,...,...
11701,trust enough share pretty humiliating experien...,joy
4770,feel rejected someone needs sleep:crying_face:,sadness
4583,wonder ever feel pain sadness always seem live...,joy
3238,im feeling exactly thrilled standing front mir...,joy


In [49]:
train['Text'] = train['Text'].str.replace('[^\w\s]',' ')
validate['Text'] = validate['Text'].str.replace('[^\w\s]',' ')
train

,Text,Emotion
3188,give charity feel valuable person receiving it...,joy
8214,im sorry havent feeling well star struck,joy
7970,got feel carefree ice cold air nipping face sm...,joy
15378,feel blank freaks out loudly_crying_face,sadness
14310,remember cammie couple months old looking swee...,anger
...,...,...
11701,trust enough share pretty humiliating experien...,joy
4770,feel rejected someone needs sleep crying_face,sadness
4583,wonder ever feel pain sadness always seem live...,joy
3238,im feeling exactly thrilled standing front mir...,joy


In [50]:
train['Text'] = train['Text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
validate['Text'] = validate['Text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
train

,Text,Emotion
3188,give charity feel valuable person receiving it...,joy
8214,im sorry havent feeling well star struck,joy
7970,got feel carefree ice cold air nipping face sm...,joy
15378,feel blank freak out loudly_crying_face,sadness
14310,remember cammie couple month old looking sweet...,anger
...,...,...
11701,trust enough share pretty humiliating experien...,joy
4770,feel rejected someone need sleep crying_face,sadness
4583,wonder ever feel pain sadness always seem live...,joy
3238,im feeling exactly thrilled standing front mir...,joy


In [51]:
def de_repeat(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

train['Text'] = train['Text'].apply(lambda x: " ".join(de_repeat(x) for x in x.split()))
validate['Text'] = validate['Text'].apply(lambda x: " ".join(de_repeat(x) for x in x.split()))
train


,Text,Emotion
3188,give charity feel valuable person receiving it...,joy
8214,im sorry havent feeling well star struck,joy
7970,got feel carefree ice cold air nipping face sm...,joy
15378,feel blank freak out loudly_crying_face,sadness
14310,remember cammie couple month old looking sweet...,anger
...,...,...
11701,trust enough share pretty humiliating experien...,joy
4770,feel rejected someone need sleep crying_face,sadness
4583,wonder ever feel pain sadness always seem live...,joy
3238,im feeling exactly thrilled standing front mir...,joy


In [52]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_texts = train['Text']
tokenizer = Tokenizer(15212,lower=True,oov_token='UNK')
tokenizer.fit_on_texts(train_texts)

print('Found %d unique words.' % len(tokenizer.word_index))

# texts_to_sequences: Transforms each text in texts to a sequence of integers. 
# It basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary.

train_texts_sequences = tokenizer.texts_to_sequences(train_texts)

# pad_sequences: Ensure that all sequences in a list have the same length. 
train_texts_pad_sequences = pad_sequences(train_texts_sequences, maxlen=80, padding='post') 

Found 12861 unique words.


In [53]:
from tensorflow.keras.utils import to_categorical
emotions = {'sadness': 0, 'joy': 1, 'surprise': 2, 'love': 3, 'anger': 4, 'fear': 5}

# Step 1: Replace all emotion values with integers
train['Emotion'] = train.Emotion.replace(emotions)
train_emotion_integers = train['Emotion'].values

# Step 2: Changing the integers to binary
train_emotion_categorical = to_categorical(train_emotion_integers)
train_emotion_categorical[:6] 

array([[0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [54]:
validate_texts = validate['Text']
validate_emotion_integers = validate.Emotion.replace(emotions)
validate_texts_sequences = tokenizer.texts_to_sequences(validate_texts)
validate_texts_pad_sequences = pad_sequences(validate_texts_sequences, maxlen=80, padding='post')
validate_emotion_categorical = to_categorical(validate_emotion_integers.values)
validate_emotion_categorical[:6]

array([[0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)

In [55]:
import tensorflow as tf
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  print("All devices: ", tf.config.list_logical_devices('TPU'))
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
  tpu_strategy = tf.distribute.get_strategy() 

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Bidirectional,Dense,Embedding,Dropout

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    model=Sequential()
    model.add(Embedding(15212,64,input_length=80))
    model.add(Dropout(0.6))
    model.add(Bidirectional(LSTM(80,return_sequences=True)))
    model.add(Bidirectional(LSTM(160)))
    model.add(Dense(len(emotions),activation='softmax'))
    print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 80, 64)            973568    
                                                                 
 dropout_4 (Dropout)         (None, 80, 64)            0         
                                                                 
 bidirectional_8 (Bidirectio  (None, 80, 160)          92800     
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 320)              410880    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 6)                 1926      
                                                                 
Total params: 1,479,174
Trainable params: 1,479,174
No

In [60]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [61]:
hist=model.fit(train_texts_pad_sequences, train_emotion_categorical, epochs=10, validation_data = (validate_texts_pad_sequences, validate_emotion_categorical))

Epoch 1/10
450/450 [==============================] - 242s 517ms/step - loss: 1.2473 - accuracy: 0.5415 - val_loss: 0.6606 - val_accuracy: 0.6906
Epoch 2/10
450/450 [==============================] - 241s 535ms/step - loss: 0.3714 - accuracy: 0.8747 - val_loss: 0.2777 - val_accuracy: 0.9075
Epoch 3/10
450/450 [==============================] - 232s 515ms/step - loss: 0.1975 - accuracy: 0.9333 - val_loss: 0.1208 - val_accuracy: 0.9663
Epoch 4/10
290/450 [==================>...........] - ETA: 1:17 - loss: 0.1304 - accuracy: 0.9551